In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [10]:
# loading dataset
batch_size = 128

transform = transforms.Compose([
                                transforms.ToTensor()])
train_set = torchvision.datasets.CIFAR100(root='./data', 
                                         train=True, 
                                         download=True,
                                         transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, 
                                          batch_size=128, 
                                          shuffle=True, 
                                          )

val_set = torchvision.datasets.CIFAR100(root='./data', 
                                        train=False, 
                                        download=True,
                                        transform=transform)

val_loader = torch.utils.data.DataLoader(val_set, 
                                         batch_size=128, 
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


 # Network architecture
 ______

 Each feature extraction layer has a conv layer followed by a batchnorm layer. CIFAR100 dataset has 100 classes for classification. The experiments folder has different ipynb files corresponding to different experiments. The architecture in this ipynb contains 7 conv layer and also has a skip connection. Later, the last layer is followed by 2 fc layers with dropout layers to reduce over fitting. During training I use an adam optimizer with learning rate scheduler that divides the learning rate by 10 (gamma = 0.1) at the 10$^{th}$, 20$^{th}$ and 30$^{th}$ epoch.

 # Experiments conducted
 _______

 - ## Changed optimizer
    - Changing the optimizer from stochastic gradient descent to adam increased the accuracy by 5 % for the same architecture on validation set (49 %). 
 - ## Removed drop out layer
    - By removing the drop out layer there is a reduction in validation accuracy as seen by the network. 
    - The accuracy on validation set reduces by 5 %.
 - ## Removed batchnorm layer
    - Removing batch norm layer incorporates bias in the network and this reduces the validation accuracy to 39 %.
 - ## Varying fc layers
    - Varying fc layers to 2 and 3 increases over fitting of the network. To reduce this I tried adding dropout layers.

# Faster RCNN
______

 - Added backbone layer to Faster RCNN and trained for multi class classification and bbox regression. The .ipynb is present in Faster RCNN folder.


 

In [0]:

# Network descriptions


class Feature_extraction_layer(nn.Module):
    '''
    Defining feature extraction layer
    '''
    def __init__(self, in_channels = 3, out_channels = 16, is_pool = True):
        
        super(Feature_extraction_layer, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding = 1)
        
        self.norm = nn.BatchNorm2d(out_channels)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.is_pool = is_pool
    
    def forward(self, input_tensor, residual = None):
        
        # appending residual connection

        if residual is None:
            out = self.norm(self.conv1(input_tensor))
        
        else:
            out = torch.cat((input_tensor, residual), dim = 1)
            out = self.norm(self.conv1(out))
        
        # If is_pool true then maxpool 2D
        if self.is_pool:
            out = self.pool(out)
        
        out = F.relu(out)
        return out


class Feature_extraction_net(nn.Module):
    '''
    Defining Feature extraction network

    '''
    def __init__(self):
        
        super(Feature_extraction_net, self).__init__()
        self.dont_pool = False
        
        self.layer1 = Feature_extraction_layer(3, 16)
        self.layer2 = Feature_extraction_layer(16, 32, self.dont_pool)
        
        # layer 1 + layer 2 concat 16 + 32 = 48

        self.layer3 = Feature_extraction_layer(48, 64, self.dont_pool)
        self.layer4 = Feature_extraction_layer(64, 64)
        self.layer5 = Feature_extraction_layer(64, 128)
        # self.layer6 = Feature_extraction_layer(128, 256)

    def forward(self, input_tensor):

        out1 = self.layer1(input_tensor)
        out2 = self.layer2(out1)
        
        out3 = self.layer3(out2, out1)
        out4 = self.layer4(out3)
        out5 = self.layer5(out4)
        # out6 = self.layer6(out5)
        # out7 = self.layer7(out6, out5)
        
        # print(out6.size())
        return out5

class CNN_network(nn.Module):

    def __init__(self):
        
        super(CNN_network, self).__init__()

        self.feature_extractor = Feature_extraction_net()
        self.drop1 = nn.Dropout(0.2)
        self.drop2 = nn.Dropout(0.1)
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        # self.fc2 = nn.Linear(512, 254)
        self.fc3 = nn.Linear(128, 100)
    
    def forward(self, input_tensor):

        out = self.feature_extractor(input_tensor)
        out = self.drop2(out.view( -1, 128 * 4 * 4 ))
        out = (F.elu(self.fc1(out)))
        # out = self.drop2(F.elu(self.fc2(out)))
        out = self.drop2(F.elu(self.fc3(out)))
        # print(out.size())
        return out

def correct_prediction( prediction, labels):

    _, index = torch.max(prediction, 1)
    correct = torch.eq(labels, index)
    total_correct = torch.sum(correct).item()
    return total_correct

def train_network(net, nepochs, lr_scheduler, train_loader, val_loader, optimizer, criterion):
    '''
    Function to train the network given different loaders
    '''
    train_loss = []
    val_loss = []
    train_acc = []
    val_acc = []

    num_train_samples = len(train_loader.dataset) 
    num_val_samples = len(val_loader.dataset)
    
    for epoch in range(nepochs):

        running_loss_train = 0.0
        correct_train = 0;
        net.train()
        
        
        for i, data in enumerate(train_loader, 0):
          
            # get the inputs
            inputs, labels = data
            
            if use_cuda and torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            # print(labels.size())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss_train += loss.item()
            correct_train += correct_prediction(outputs, labels) 

        mean_loss = running_loss_train / num_train_samples
        accuracy_train = round((correct_train / num_train_samples)*100,2)
        
        if epoch % 2 == 0:
        
            print("Epoch ", epoch, " train loss ", mean_loss)
            print("Train accuracy: ", accuracy_train,"%" )
                
        
        train_acc.append(accuracy_train)
        train_loss.append(mean_loss)

        # check for val set
        net.eval()
        running_loss_val = 0
        correct_val = 0

        for i, data in enumerate(val_loader, 0):
          
            # get the inputs
            inputs, labels = data
            
            if use_cuda and torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            # forward + backward + optimize
            outputs = net(inputs)
            # print(labels.size())
            loss = criterion(outputs, labels)

            # print statistics
            running_loss_val += loss.item()
            correct_val += correct_prediction(outputs, labels) 

        mean_loss_val = running_loss_val / num_val_samples
        accuracy_val = round((correct_val / num_val_samples)*100,2)
        
        if epoch % 2 == 0:
            print("Epoch ", epoch, " validation loss ", mean_loss_val)
            print("Validation accuracy: ", accuracy_val ,"%" )

        val_acc.append(accuracy_val)
        val_loss.append(mean_loss_val)
        lr_scheduler.step()

    return train_loss, train_acc, val_loss, val_acc

def weights_init(net):
    if isinstance(net, nn.Conv2d):
        nn.init.xavier_normal_(net.weight.data)
        nn.init.xavier_normal_(net.bias.data)

In [48]:
if __name__=='__main__':
    
    net = CNN_network()
    # net.apply(weights_init)
    use_cuda = True

    if use_cuda and torch.cuda.is_available():
        net.cuda()
        print('using gpu')

    criterion = nn.CrossEntropyLoss()
    lr = 1e-3
    momentum = 0.99
    nepochs = 100
    optimizer = optim.Adam(net.parameters(), lr)
    
    # lr_scheduler
    lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [10, 20, 30], gamma=0.1)

    train_network(net, 50, lr_scheduler, train_loader, val_loader, optimizer, criterion)

using gpu
Epoch  0  train loss  0.02842044912338257
Train accuracy:  16.94 %
Epoch  0  validation loss  0.02342667770385742
Validation accuracy:  27.61 %
Epoch  2  train loss  0.020096738901138306
Train accuracy:  38.31 %
Epoch  2  validation loss  0.019077400922775267
Validation accuracy:  39.12 %
Epoch  4  train loss  0.016921128940582276
Train accuracy:  48.05 %
Epoch  4  validation loss  0.01712595282793045
Validation accuracy:  43.56 %
Epoch  6  train loss  0.014975869448184968
Train accuracy:  54.31 %
Epoch  6  validation loss  0.01595147725343704
Validation accuracy:  48.81 %
Epoch  8  train loss  0.013350130441188813
Train accuracy:  59.7 %
Epoch  8  validation loss  0.015801086938381195
Validation accuracy:  49.03 %
Epoch  10  train loss  0.010171977208852767
Train accuracy:  71.11 %
Epoch  10  validation loss  0.014444028282165528
Validation accuracy:  54.21 %
Epoch  12  train loss  0.009411103267669677
Train accuracy:  73.78 %
Epoch  12  validation loss  0.014407230508327484

In [0]:
 from google.colab import files

torch.save(net.state_dict(), 'checkpointcv_4_xavier.pth')

# download checkpoint file
files.download('checkpointcv_4_xavier.pth')